## 참고한 코드

쏘니님의 코드 [Private 1위, Public 점수 0.7829, LGBM + NN Ensemble]  
https://dacon.io/competitions/official/235647/codeshare/1812

## 라이브러리

In [1]:
# Colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import copy
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

## 데이터 로드

In [3]:
train = pd.read_csv('./drive/MyDrive/data/train.csv')
test = pd.read_csv('./drive/MyDrive/data/test.csv')
pd.set_option('display.max_columns', None)

In [4]:
x_train = train.copy()
dataset = [x_train, test]

## 이상치 처리

In [5]:
intro_error_big = x_train['introelapse'].quantile(.95)
intro_error_small = x_train['introelapse'].quantile(.003)
survey_error_big = x_train['surveyelapse'].quantile(.95)
survey_error_small = x_train['surveyelapse'].quantile(.003)
test_error_big = x_train['testelapse'].quantile(.95)
test_error_small = x_train['testelapse'].quantile(.003)
print(intro_error_big,intro_error_small,survey_error_big,survey_error_small,test_error_big,test_error_small)

976.0 1.0 381.0 60.0 290.0 42.997


In [6]:
index = x_train[x_train['introelapse'] > intro_error_big].index
x_train['introelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['surveyelapse'] > survey_error_big].index
x_train['surveyelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['testelapse'] > test_error_big].index
x_train['testelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['introelapse'] < intro_error_small].index
x_train['introelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['surveyelapse'] < survey_error_small].index
x_train['surveyelapse'].iloc[index] = np.nan
print(index)
index = x_train[x_train['testelapse'] < test_error_small].index
x_train['testelapse'].iloc[index] = np.nan
print(index)

Int64Index([   35,    56,    59,    96,   113,   130,   142,   181,   212,
              227,
            ...
            14796, 14801, 14823, 14862, 14867, 14869, 14892, 14924, 14940,
            14962],
           dtype='int64', length=748)
Int64Index([   43,    48,    60,    74,    89,    90,    92,    97,   112,
              113,
            ...
            14875, 14878, 14887, 14892, 14906, 14939, 14949, 14974, 14984,
            14994],
           dtype='int64', length=744)
Int64Index([    0,     4,    13,    15,    24,    55,    60,    73,    79,
               87,
            ...
            14837, 14839, 14874, 14875, 14878, 14889, 14894, 14898, 14926,
            14991],
           dtype='int64', length=749)
Int64Index([], dtype='int64')
Int64Index([    0,   548,   632,   663,   738,   835,  1775,  1858,  2670,
             2883,  3176,  3712,  3833,  4115,  4766,  4820,  5099,  5467,
             5578,  5692,  6328,  6575,  6811,  7588,  7746,  8799,  9142,
             935

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
index = test[test['introelapse'] > intro_error_big].index
test['introelapse'].iloc[index] = np.nan
print(index)
index = test[test['surveyelapse'] > survey_error_big].index
test['surveyelapse'].iloc[index] = np.nan
print(index)
index = test[test['testelapse'] > test_error_big].index
test['testelapse'].iloc[index] = np.nan
print(index)
index = test[test['introelapse'] < intro_error_small].index
test['introelapse'].iloc[index] = np.nan
print(index)
index = test[test['surveyelapse'] < survey_error_small].index
test['surveyelapse'].iloc[index] = np.nan
print(index)
index = test[test['testelapse'] < test_error_small].index
test['testelapse'].iloc[index] = np.nan
print(index)

Int64Index([    3,    57,    84,    92,   102,   112,   113,   158,   169,
              170,
            ...
            35218, 35232, 35245, 35246, 35272, 35335, 35368, 35414, 35427,
            35429],
           dtype='int64', length=1890)
Int64Index([   13,    63,    94,   127,   284,   303,   304,   320,   328,
              333,
            ...
            35261, 35286, 35287, 35315, 35365, 35366, 35368, 35385, 35398,
            35425],
           dtype='int64', length=1782)
Int64Index([   33,   151,   164,   204,   244,   263,   266,   278,   280,
              284,
            ...
            35228, 35246, 35269, 35300, 35346, 35351, 35385, 35391, 35435,
            35445],
           dtype='int64', length=1821)
Int64Index([21768, 33952], dtype='int64')
Int64Index([  569,   657,   681,  2307,  2504,  3137,  3372,  4337,  4383,
             4969,  5351,  5805,  6029,  6218,  6297,  6413,  6441,  6937,
             8133,  9442, 10077, 10098, 10284, 10364, 10462, 10782, 11341,
 

In [8]:
index = x_train[x_train['familysize'] > 23].index
x_train['familysize'].iloc[index] = np.nan
print(index)

index = test[test['familysize'] > 23].index
test['familysize'].iloc[index] = np.nan
print(index)

Int64Index([1019, 1287, 9817], dtype='int64')
Int64Index([4665, 5125, 6475, 10075, 11773, 25389, 30771], dtype='int64')


In [9]:
index = x_train[x_train['age'] > 100].index
x_train['age'].iloc[index] = np.nan
print(index)

index = test[test['age'] > 100].index
test['age'].iloc[index] = np.nan
print(index)

Int64Index([1739, 4002, 4976, 6725, 9273, 14135], dtype='int64')
Int64Index([ 2578,  2802,  4609,  8190,  8625, 11922, 13943, 17311, 24301,
            26070, 26332, 28448, 30699, 33783, 33787, 34046],
           dtype='int64')


In [10]:
#index 피처 드랍
for data in dataset:
    data.drop('index', axis=1, inplace = True)

## 결측치 처리

In [11]:
#country피처의 결측값을 x_train의 최빈값으로
for data in dataset:
    data['country'] = data['country'].fillna(x_train['country'].mode()[0])

In [12]:
#country피처 원핫 인코딩
country_list = ['USA','GBR','CAN','AUS','DEU','PHL','BRA','POL','IND','FRA','ITA','NLD']
enc = OneHotEncoder(categories=[country_list],handle_unknown='ignore')

temp = enc.fit_transform(x_train['country'].values.reshape(-1,1)).toarray()
temp = pd.DataFrame(temp,columns=country_list)
x_train[country_list] = temp.copy()

temp = enc.transform(test['country'].values.reshape(-1,1)).toarray()
temp = pd.DataFrame(temp,columns=country_list)
test[country_list] = temp.copy()

x_train.drop('country', axis=1, inplace = True)  
test.drop('country', axis=1, inplace = True)  

In [13]:
#knn을 이용하여 나머지 피처들 결측치 처리
imputer = KNNImputer(n_neighbors=5)
column_list1 = x_train.columns.tolist()
column_list2 = column_list1.copy()
column_list2.remove('nerdiness')
print(column_list1)
print(column_list2)
x_train[column_list1] = imputer.fit_transform(x_train[column_list1])
imputer.fit(x_train[column_list2])
test[column_list2] = imputer.transform(test[column_list2])

['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'introelapse', 'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10', 'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education', 'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation', 'voted', 'married', 'familysize', 'ASD', 'nerdiness', 'USA', 'GBR', 'CAN', 'AUS', 'DEU', 'PHL', 'BRA', 'POL', 'IND', 'FRA', 'ITA', 'NLD']
['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'introelapse', 'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3',

## 피처엔지니어링

In [14]:
Answers1 = ['Q1','Q2','Q3','Q4','Q5',
             'Q6','Q7','Q8','Q9','Q10', 
             'Q11','Q12','Q13','Q14','Q15', 
             'Q16','Q17','Q18','Q19','Q20',
             'Q21','Q22','Q23','Q24','Q25',
             'Q26']

Answers1_1 = ['Q1','Q2','Q3','Q4','Q5',
             'Q6','Q7','Q8','Q9','Q10', 
             'Q11','Q12','Q13','Q14','Q15', 
             'Q16','Q17','Q18','Q19','Q20']

Answers1_2 = ['Q21','Q22','Q23','Q24','Q25',
             'Q26']

Answers2 = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5',
            'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']
            
Answers3 = ['VCL1','VCL2','VCL3','VCL4','VCL5',
            'VCL6','VCL7','VCL8','VCL9','VCL10',
            'VCL11','VCL12','VCL13','VCL14','VCL15',
            'VCL16']

In [16]:
#Q로 T/V/M 나누기
for data in dataset:
  data['T'] = data['Q1'] + data['Q2'] + data['Q3'] + data['Q15']
  data['V'] = data['Q4'] + data['Q5'] + data['Q7'] + data['Q8'] + data['Q10'] + data['Q11'] + data['Q12']  + data['Q13'] + data['Q17'] + data['Q19'] + data['Q20'] 
  data['M'] = data['Q9'] + data['Q16']

In [17]:
#Q질문 이용하여 점수 계산(Q26개 설문의 답)
Ancoms = list(combinations(Answers1, 2))
for data in dataset:
    data['Q_score'] = data[Answers1].mean(axis = 1)
    data['Q_var'] = data[Answers1].var(axis=1)
    data['Q_score_1'] = data[Answers1_1].mean(axis = 1)
    data['Q_var_1'] = data[Answers1_1].var(axis=1)
    data['Q_score_2'] = data[Answers1_2].mean(axis = 1)
    data['Q_var_2'] = data[Answers1_2].var(axis=1)
    temp_dict = {}
    for a,b in Ancoms:
        temp_dict['%s_dv_%s'%(a,b)] = data[a]/data[b]
    data[list(temp_dict.keys())] = pd.DataFrame(temp_dict).copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


In [18]:
#해당 column의 값을 뒤집는다 1~5 -> 5~1
flipping_columns = ["TIPI6", "TIPI2", "TIPI8", "TIPI4", "TIPI10"]

for data in dataset:
    for flip in flipping_columns:
        data[flip] = 6 - data[flip]

In [19]:
# TIPI test에 따라 항목 나누기  외향적 1~5   내향적 1~5 -> 5~1   내향적이 낮다 -> 외향적이 높다로 변환
# 외향성, 친화성, 양심성, 정서적 안정, 개방성
for data in dataset:
    data['Ex'] = (data['TIPI1']+data['TIPI6'])/2
    data['Ag'] = (data['TIPI7']+data['TIPI2'])/2
    data['Con'] = (data['TIPI3']+data['TIPI8'])/2
    data['Es'] = (data['TIPI9']+data['TIPI4'])/2
    data['Op'] = (data['TIPI5']+data['TIPI10'])/2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1

In [20]:
for data in dataset:
    temp = data[data['gender'] == 3.0].copy()
    temp['gender'] = 1.5
    data[data['gender'] == 3.0] = temp.copy()

In [21]:
for data in dataset:
    data['Es_gender'] = data['Es']*data['gender']
    data['Con_gender'] = data['Con']*data['gender']
    data['Op_gender'] = data['Op']*data['gender']

In [22]:
#VLC질문을 이용하여 점수계산(단어의 정의 알고있는지 유무)
for data in dataset:
    data['VLC_score'] = data[Answers3].sum(axis=1)       
    data['VLC_var'] = data[Answers3].var(axis=1)
    data['VLC_mean'] = data[Answers3].mean(axis = 1)   

In [23]:
#지연시간
for data in dataset:
    data['delay'] = data[['introelapse','testelapse','surveyelapse']].sum(axis=1)
    data['delay'] = data['delay'] ** (1/10)
    data['delay_var'] = data['delay'].var()
    data['introelapse_delay_var'] = data['introelapse'].var()
    data['testelapse_delay_var'] = data['testelapse'].var()
    data['surveyelapse_delay_var'] = data['surveyelapse'].var()
    data['introelapse'] = data['introelapse'] ** (1/10)
    data['testelapse'] = data['introelapse'] ** (1/10)
    data['surveyelapse'] = data['introelapse'] ** (1/10)

In [24]:
# 13~18,19~29,30이상 피처 생성
def age_grouping1(age):
    if (age <= 18 ):
        age = '18low'
    elif (age > 18 and age<=29):
        age = '18upper29low'
    else:
        age = '30upper'
    return age
for data in dataset:
  data['age_group1'] = data['age'].apply(age_grouping1)

In [25]:
#age로 age_group object 생성
def age_grouping2(age):
    age = (age//10.)*10
    if(age >= 70):
        age = '+70s'
    else:
        age = str(int(age))+'s'
    return age
for data in dataset:
  data['age_group2'] = data['age'].apply(age_grouping2)

In [26]:
#age_group1, age_group2의 라벨 인코딩
encoder = LabelEncoder()
x_train['age_group1'] = encoder.fit_transform(x_train['age_group1'])
test['age_group1'] = encoder.transform(test['age_group1'])
x_train['age_group2'] = encoder.fit_transform(x_train['age_group2'])
test['age_group2'] = encoder.transform(test['age_group2'])

In [27]:
#age 피처 드랍
for data in dataset:
    data.drop('age', axis=1, inplace = True)

In [28]:
y_train = x_train['nerdiness']
x_train.drop('nerdiness', axis=1, inplace = True)

In [29]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Columns: 430 entries, Q1 to age_group2
dtypes: float64(428), int64(2)
memory usage: 49.2 MB


In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Columns: 430 entries, Q1 to age_group2
dtypes: float64(428), int64(2)
memory usage: 116.3 MB


## Extra Trees

In [31]:
def ex_rfe_4040(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=4040)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [32]:
ex_archive_4040 = ex_rfe_4040(x_train, y_train)

430 0.8626600946615376
387 0.8614962020421928
348 0.8647778346670294
313 0.8614326698420282
281 0.8617506189707438
252 0.8646942775783772
226 0.8640004656146733
203 0.8653811744726755
182 0.8644286236620421
163 0.8649354406239006
146 0.8642345695268451
131 0.8643688371244721
117 0.8624504816201777
105 0.8602975900118738
94 0.8615155066109503
84 0.8638618760986677
75 0.8647264038383246
67 0.8618894966146413
60 0.8637784630739616
54 0.8646389570231318
48 0.8641857318491674
43 0.8642887375705228


In [33]:
model1 = ExtraTreesClassifier()

x_train_1 = x_train[ex_archive_4040.iloc[9,2]]

model1.fit(x_train_1, y_train)

pred_y1 = model1.predict_proba(test[ex_archive_4040.iloc[9,2]])
pred_y1 = pred_y1[:,1]

In [34]:
def ex_rfe_1234(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [35]:
ex_archive_1234 = ex_rfe_1234(x_train, y_train)

430 0.856983568614652
387 0.8597461225699983
348 0.8572435770728276
313 0.8617720337775407
281 0.8579885515833796
252 0.860820201709103
226 0.8587243324246924
203 0.8574758608722034
182 0.8575501284815277
163 0.859644273952956
146 0.862032473188962
131 0.8607578571395736
117 0.8621114812839878
105 0.8600344302913014
94 0.8610688617318114
84 0.8609677313701785
75 0.8602391330829591
67 0.8624612716683292
60 0.8610996030633306
54 0.8618225989583572
48 0.8615814087919058
43 0.8603543412506148


In [36]:
model2 = ExtraTreesClassifier()

x_train_2 = x_train[ex_archive_1234.iloc[8,2]]

model2.fit(x_train_2, y_train)

pred_y2 = model2.predict_proba(test[ex_archive_1234.iloc[8,2]])
pred_y2 = pred_y2[:,1]

In [37]:
def ex_rfe_99087(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [38]:
ex_archive_99087 = ex_rfe_99087(x_train, y_train)

430 0.8638318571655278
387 0.864447949272879
348 0.8654945583801177
313 0.862381180182936
281 0.8644107192551926
252 0.8640487687357548
226 0.8658491778922118
203 0.8658887078337708
182 0.8648790412537345
163 0.8635482478029977
146 0.8634805437940004
131 0.8646586797976987
117 0.8662278028597254
105 0.8634621444030203
94 0.8659480746187301
84 0.8622692026393927
75 0.8647453581786443
67 0.8667329236402276
60 0.8649155525452108
54 0.8643388466341764
48 0.8654702654342141
43 0.8658463029873711


In [39]:
model3 = ExtraTreesClassifier()

x_train_3 = x_train[ex_archive_99087.iloc[11,2]]

model3.fit(x_train_3, y_train)

pred_y3 = model3.predict_proba(test[ex_archive_99087.iloc[11,2]])
pred_y3 = pred_y3[:,1]

In [40]:
def ex_rfe_42(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = ExtraTreesClassifier()
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [41]:
ex_archive_42 = ex_rfe_42(x_train, y_train)

430 0.875744009928575
387 0.8699296514867535
348 0.8720009348333648
313 0.8724615877891422
281 0.875648886028616
252 0.872946273560794
226 0.8746190726729327
203 0.874992228909837
182 0.8744506990527328
163 0.8740042930955879
146 0.8763970693204268
131 0.8728026523018544
117 0.8723253058931342
105 0.8743286641553573
94 0.8764667213137403
84 0.8777354237373993
75 0.8735226294145547
67 0.8776297944748125
60 0.876868227638832
54 0.8748676036490737
48 0.8759211620024775
43 0.8731701960847521


In [42]:
model4 = ExtraTreesClassifier()

x_train_4 = x_train[ex_archive_42.iloc[11,2]]

model4.fit(x_train_4, y_train)

pred_y4 = model4.predict_proba(test[ex_archive_42.iloc[11,2]])
pred_y4 = pred_y4[:,1]

In [43]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)
submission = pd.DataFrame()

submission['index'] = submission.index
submission['nerdiness'] = pred_all
submission['index'] = submission.index

submission.to_csv('./drive/MyDrive/submission.csv', index=False)